## Back bone networks

* Resnet
* Resnet Variants
* ResNext
* DenseNet
* MobileNet
* SqueezeNet

## ResNet

Deep Residual Learning for Image Recognition

https://arxiv.org/abs/1512.03385


![image info](./pictures/residual.png)


*Philosphy*

* Philosophy borrowed from VGG nets. (https://arxiv.org/abs/1409.1556)





The convolutional layers mostly have 3×3 filters 

we use very small3×3receptive fields throughout the whole net,which are convolved with the input at every pixel (with stride1). It is easy to see that a stack of two3×3conv. layers (without spatial pooling in between) has an effective receptive field of5×5; 

such layers have a7×7effective receptive field. So what have we gained by using, for instance, astack of three3×3conv. layers instead of a single7×7layer? First, we incorporate three non-linearrectification layers instead of a single one, which makes thedecision function more discriminative.Second, we decrease the number of parameters:  assuming thatboth the input and the output of athree-layer3×3convolution stack hasCchannels, the stack is parametrised by3(32C2)= 27C2weights; at the same time, a single7×7conv. layer would require72C2= 49C2parameters, i.e.81%more. This can be seen as imposing a regularisation on the7×7conv. filters, forcing them tohave a decomposition through the3×3filters (with non-linearity injected in between)


and follow  two  simple  design  rules:  




(i)  for  the  same  output feature map size,  the layers have the same number of filters;  and  
(ii)  if  the  feature  map  size  is  halved,  the  number  of  filters  is  doubled  so  as  to  preserve  the  time  complexity  per  layer.   

We  perform  downsampling  directly  by convolutional layers that have a stride of 2.  The network ends with a global average pooling layer and a 1000-wayfully-connected  layer  with  softmax.   The  total  number  ofweighted layers is 34


In [30]:
import tensorflow as tf
from tensorflow.keras import layers,Input, Model



def conv_block(x, no_filters, kernel_size = (3,3) ,strides =(1,1), padding ='valid',name = None):
    
    x = layers.Conv2D(filters = no_filters, kernel_size = kernel_size , padding ='same', \
                     strides = strides, activation = None, use_bias = True, \
                     kernel_initializer = 'glorot_uniform', bias_initializer = 'zero', \
                     name = name)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    return x



def model_plain(input_img):
    
    # 34 Layer Plain
    
    x = conv_block(input_img, no_filters = 64,kernel_size = (7,7), strides = (2,2),name="conv_start_1")
    x = layers.MaxPool2D(pool_size=(3, 3), strides=(2,2), padding='same')(x)
    
    # Block A #
    name = 'conv_block_a_'
    for i in range(0,6):
        x = conv_block(x, no_filters = 64 ,name = name + str(i))
    
    # Block B #
    name = 'conv_block_b_'
    x = conv_block(x,no_filters=128, strides=(2,2), name =name + '_1')
    for i in range(0,7):
        x = conv_block(x, no_filters = 128, name = name +  str(i + 2))

        
    # Block c #
    name = 'conv_block_c_'
    x = conv_block(x, no_filters=256, strides=(2,2), name =name + '_1')
    for i in range(0,11):
        x = conv_block(x, no_filters = 256, name = name +  str(i + 2))

    # Block D #
    name = 'conv_block_d_'
    x = conv_block(x,no_filters=512, strides=(2,2), name =name + '_1')
    for i in range(0,5):
        x = conv_block(x, no_filters = 512, name = name +  str(i + 2))


    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1000)(x)
    

    
    return x



img_input = Input((224,224,3))
x = model_plain(img_input)


model = Model(img_input, x)

model.summary()

    
# Plot the model
# tf.keras.utils.plot_model(model,show_shapes=True, show_layer_names=True, rankdir ='TB')
# rankdir specifies 'TB'. top to bottom plot or 'LR' left to right plot


Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv_start_1 (Conv2D)        (None, 112, 112, 64)      9472      
_________________________________________________________________
batch_normalization_337 (Bat (None, 112, 112, 64)      256       
_________________________________________________________________
re_lu_335 (ReLU)             (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 56, 56, 64)        0         
_________________________________________________________________
conv_block_a_0 (Conv2D)      (None, 56, 56, 64)        36928     
_________________________________________________________________
batch_normalization_338 (Bat (None, 56, 56, 64)        256

## Adding shortcuts

The  baseline  architectures are the same as the above plain nets, expect that a shortcut connection is added to each pair of 3×3 filters 

In [57]:
def identity_block(in_tensor, no_filters,name, strides = (1,1)):
    
    x = conv_block(in_tensor, no_filters,strides = strides, name = name + "_conv_1")
    
    x = layers.Conv2D(filters = no_filters, kernel_size = (3,3) , padding ='same', \
                     strides = (1,1), activation = None, use_bias = True, \
                     kernel_initializer = 'glorot_uniform', bias_initializer = 'zero', \
                     name = name + "_conv_2")(x)
    
    x = layers.BatchNormalization()(x)
    
    x = layers.add([x, in_tensor])
    x = layers.Activation('relu')(x)
    
    return x


def convoluted_block(in_tensor, no_filters,name, strides = (1,1)):
    
    x = conv_block(in_tensor, no_filters,strides = strides, name = name + "_conv_1")
    
    x = layers.Conv2D(filters = no_filters, kernel_size = (3,3) , padding ='same', \
                     strides = (1,1), activation = None, use_bias = True, \
                     kernel_initializer = 'glorot_uniform', bias_initializer = 'zero', \
                     name = name + "_conv_2")(x)
    
    x = layers.BatchNormalization()(x)
    
    shortcut = layers.Conv2D(no_filters, (1, 1), strides=strides,
                             kernel_initializer='glorot_uniform',
                             name= name + "_shortcut")(in_tensor)
    shortcut = layers.BatchNormalization()(shortcut)

    
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    
    return x




def model_resnet(input_img):
    
    # 34 Layer Residual Net
    x = conv_block(input_img, no_filters = 64,kernel_size = (7,7), strides = (2,2),name="conv_1")
    x = layers.MaxPool2D(pool_size=(3, 3), strides=(2,2), padding='same')(x)
    
    # Block A #
    name = 'conv_block_2_'
    for i in range(0,3):
        x = identity_block(x, no_filters = 64 ,name = name + 'identity_block_' + str(i+1) + '_')
    
    
    # Block B #
    name = 'conv_block_3_'
    x = convoluted_block(x,no_filters=128, strides=(2,2), name = name + 'convoluted_block' + '_1')
    for i in range(0,3):
        x = identity_block(x, no_filters = 128, name = name  + 'identity_block_' +  str(i + 2))

        
    # Block c #
    name = 'conv_block_4_'
    x = convoluted_block(x, no_filters=256, strides=(2,2), name =name + 'convoluted_block' + '_1')
    for i in range(0,5):
        x = identity_block(x, no_filters = 256, name = name  + 'identity_block_' +  str(i + 2))

    # Block D #
    name = 'conv_block_5_'
    x = convoluted_block(x,no_filters=512, strides=(2,2), name =name + 'convoluted_block' + '_1')
    for i in range(0,2):
        x = identity_block(x, no_filters = 512, name = name + 'identity_block_'  +  str(i + 2))


    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1000)(x)
    

    
    return x

img_input = Input((224,224,3), name="input_layer")
x = model_resnet(img_input)


model = Model(img_input, x)

#tf.keras.utils.plot_model(model,show_shapes=False, show_layer_names=True, rankdir ='TB', expand_nested =True)


## Resnet Depths

![image info](./pictures/Resnet-Depths.png)



## Bottleneck architecture

In [ ]:
## References

https://www.pyimagesearch.com/2017/03/20/imagenet-vggnet-resnet-inception-xception-keras/
    
